In [1]:
from scipy.io import mmread
import pandas as pd
import numpy as np

np.random.seed(42)

In [2]:
df = pd.DataFrame(mmread('data/E-MTAB-10290.aggregated_filtered_normalised_counts.mtx').todense())
# read the gene names from file and add them as row names
with open('data/E-MTAB-10290.aggregated_filtered_normalised_counts.mtx_rows') as f:
    gene_names = [i.split()[0] for i in f.read().splitlines()]
df.index = gene_names

# read the cell names from file and add them as column names
with open('data/E-MTAB-10290.aggregated_filtered_normalised_counts.mtx_cols') as f:
    cell_names = f.read().splitlines()
df.columns = cell_names

# transpose the dataframe so that the genes are the columns and the cells are the rows
df = df.transpose()

# import the experimental design from tsv file
design = pd.read_csv('data/ExpDesign-E-MTAB-10290.tsv', sep='\t')

# keep only the columns we need
design = design[['Assay', 'Sample Characteristic[disease]']]
design.columns = ['cell', 'disease']

design.set_index('cell', inplace=True)

In [ ]:

print(design.index)
print(df.index)

design = np.ndarray.flatten(design.values)
design

Index(['SAMEA8461660-AAACCCAAGGTTCAGG', 'SAMEA8461660-AAACCCAAGGTTTACC',
       'SAMEA8461660-AAACCCAGTAGCTGAG', 'SAMEA8461660-AAACCCAGTCAGATTC',
       'SAMEA8461660-AAACCCAGTGCGAACA', 'SAMEA8461660-AAACCCAGTGTCGCTG',
       'SAMEA8461660-AAACGAAAGTAAACAC', 'SAMEA8461660-AAACGAACACCGCTGA',
       'SAMEA8461660-AAACGAAGTCTCAAGT', 'SAMEA8461660-AAACGAATCCTCAGAA',
       ...
       'SAMEA8461667-TTTGGAGCAATGCAGG', 'SAMEA8461667-TTTGGAGGTTTCGCTC',
       'SAMEA8461667-TTTGGTTAGCTGTTCA', 'SAMEA8461667-TTTGGTTCACACAGAG',
       'SAMEA8461667-TTTGGTTCACGTCATA', 'SAMEA8461667-TTTGGTTCACTTGACA',
       'SAMEA8461667-TTTGGTTTCCCATGGG', 'SAMEA8461667-TTTGTTGAGATCCAAA',
       'SAMEA8461667-TTTGTTGCAGCCGTCA', 'SAMEA8461667-TTTGTTGGTTTACTGG'],
      dtype='object', name='cell', length=29203)
Index(['SAMEA8461660-AAACCCAAGGTTCAGG', 'SAMEA8461660-AAACCCAAGGTTTACC',
       'SAMEA8461660-AAACCCAGTAGCTGAG', 'SAMEA8461660-AAACCCAGTCAGATTC',
       'SAMEA8461660-AAACCCAGTGCGAACA', 'SAMEA8461660-AAACCCAGT

array(['normal', 'normal', 'normal', ..., 'cutaneous melanoma',
       'cutaneous melanoma', 'cutaneous melanoma'], dtype=object)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

# set the seed for reproducibility
torch.manual_seed(42)
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
device

device(type='cuda', index=0)

In [ ]:
# encode the target labels into integers
le = LabelEncoder()
design = le.fit_transform(design.ravel())

le.classes_

array(['cutaneous melanoma', 'normal'], dtype=object)

In [ ]:
# define the model1
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(25511, 5000)
        self.fc2 = nn.Linear(5000, 1000)
        self.fc3 = nn.Linear(1000, 100)
        self.fc4 = nn.Linear(100, 2)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return self.fc4(x)

# define the model2
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(25511, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 2)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# define the model3
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.fc1 = nn.Linear(25511, 100)
        self.fc2 = nn.Linear(100, 2)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)


In [ ]:
# define the optimizer and the loss
criterion = nn.CrossEntropyLoss()

# define the training function
def train(model, train_dl, optimizer, criterion):
    model.train()
    running_loss = 0
    for i, (X_batch, y_batch) in enumerate(train_dl):
        optimizer.zero_grad()
        y_pred = model(X_batch.to(device))
        loss = criterion(y_pred, y_batch.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_dl)
    return model, train_loss

# define the testing function
def val(model, test_dl, criterion):
    model.eval()
    running_loss = 0
    predictions, truth = [], []
    with torch.no_grad():
        for i, (X_batch, y_batch) in enumerate(test_dl):
            y_pred = model(X_batch.to(device))
            loss = criterion(y_pred, y_batch.to(device))
            running_loss += loss.item()
            _, predicted = torch.max(y_pred.data, 1)
            predictions.extend(predicted.cpu().numpy())
            truth.extend(y_batch.cpu().numpy())
    val_loss = running_loss / len(test_dl)
    return val_loss, f1_score(y_batch.cpu().numpy(), predicted.cpu().numpy()), predictions, truth

In [ ]:
# do cv and test split
X_train, X_val_test, y_train, y_val_test = train_test_split(df, design, test_size=0.5, random_state=42, stratify=design)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.4, random_state=42, stratify=y_val_test)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)


(14601, 25511) (8761, 25511) (5841, 25511) (14601,) (8761,) (5841,)


In [ ]:
# define models
model1 = Net1()
model2 = Net2()
model3 = Net3()

# convert the data into PyTorch tensors
T_X_train = torch.from_numpy(X_train.values).float()
T_X_val = torch.from_numpy(X_val.values).float()

# convert the target to a torch tensor
T_y_train = torch.squeeze(torch.from_numpy(y_train)).long()
T_y_val = torch.squeeze(torch.from_numpy(y_val)).long()

# create Tensor datasets
train_ds = data_utils.TensorDataset(T_X_train, T_y_train)
val_ds = data_utils.TensorDataset(T_X_val, T_y_val)

# define the dataloader
train_dl = data_utils.DataLoader(train_ds, batch_size=64, shuffle=True)
val_dl = data_utils.DataLoader(val_ds, batch_size=64, shuffle=False)


# define the results dataframe
results = pd.DataFrame(columns=['model', 'train_loss', 'val_loss', 'val_f1'])

In [ ]:
model1.to(device)

# Initialize optimizer
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-4)

epochs = 2

train_losses, val_losses, val_f1s = [], [], []

for epoch in range(epochs):
    model, train_loss = train(model1, train_dl, optimizer, criterion)
    val_loss, val_f1, _, _ = val(model1, val_dl, criterion)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_f1s.append(val_f1)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f} | Val F1: {val_f1:.2f} |')
    
# final evaluation
val_loss, val_f1, predictions, truth = val(model1, val_dl, criterion)

# append results to dataframe
new_row = pd.DataFrame([[model, train_loss, val_loss, val_f1]], columns=['model', 'train_loss', 'val_loss', 'val_f1'])
results = pd.concat([results, new_row], axis=0)

# save the model
torch.save(model1.state_dict(), 'model1.pt')

| Epoch: 01 | Train Loss: 0.938 | Val Loss: 0.033 | Val F1: 1.00 |
| Epoch: 02 | Train Loss: 0.032 | Val Loss: 0.040 | Val F1: 1.00 |


In [ ]:
model2.to(device)

# Initialize optimizer
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-4)

epochs = 3

train_losses, val_losses, val_f1s = [], [], []

for epoch in range(epochs):
    model, train_loss = train(model2, train_dl, optimizer, criterion)
    val_loss, val_f1, _, _ = val(model2, val_dl, criterion)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_f1s.append(val_f1)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f} | Val F1: {val_f1:.2f} |')
    
# final evaluation
val_loss, val_f1, predictions, truth = val(model2, val_dl, criterion)

# append results to dataframe
new_row = pd.DataFrame([[model, train_loss, val_loss, val_f1]], columns=['model', 'train_loss', 'val_loss', 'val_f1'])
results = pd.concat([results, new_row], axis=0)

# save the model
torch.save(model2.state_dict(), 'model2.pt')

| Epoch: 01 | Train Loss: 0.746 | Val Loss: 0.058 | Val F1: 1.00 |
| Epoch: 02 | Train Loss: 0.044 | Val Loss: 0.041 | Val F1: 1.00 |
| Epoch: 03 | Train Loss: 0.124 | Val Loss: 0.169 | Val F1: 1.00 |


In [ ]:
model3.to(device)

# Initialize optimizer
optimizer = torch.optim.Adam(model3.parameters(), lr=1e-4)

epochs = 3

train_losses, val_losses, val_f1s = [], [], []

for epoch in range(epochs):
    model, train_loss = train(model3, train_dl, optimizer, criterion)
    val_loss, val_f1, _, _ = val(model3, val_dl, criterion)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_f1s.append(val_f1)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f} | Val F1: {val_f1:.2f} |')
    
# final evaluation
val_loss, val_acc, predictions, truth = val(model3, val_dl, criterion)

# append results to dataframe
new_row = pd.DataFrame([[model, train_loss, val_loss, val_f1]], columns=['model', 'train_loss', 'val_loss', 'val_f1'])
results = pd.concat([results, new_row], axis=0)

# save the model
torch.save(model3.state_dict(), 'model3.pt')

| Epoch: 01 | Train Loss: 2.678 | Val Loss: 0.099 | Val F1: 0.97 |
| Epoch: 02 | Train Loss: 0.103 | Val Loss: 0.076 | Val F1: 1.00 |
| Epoch: 03 | Train Loss: 0.031 | Val Loss: 0.048 | Val F1: 1.00 |


In [ ]:
results

,model,train_loss,val_loss,val_f1
0,"Net1(\n (fc1): Linear(in_features=25511, out_...",0.031809,0.039950,1.0
0,"Net2(\n (fc1): Linear(in_features=25511, out_...",0.123723,0.168616,1.0
0,"Net3(\n (fc1): Linear(in_features=25511, out_...",0.030998,0.047528,1.0


In [ ]:
# gradients function
def calculate_gradients(model, inputs, targets):
    model.eval()
    inputs.requires_grad = True
    outputs = model(inputs.to(device))
    loss = criterion(outputs, targets.to(device))
    loss.backward()

    # Access the gradients of the inputs
    gradients = inputs.grad.abs().mean(dim=0).detach().cpu().numpy()

    return gradients

def remove_least_important_features(df, importance, removed_features, num_features_to_remove=1):
    sorted_indices = np.argsort(importance)
    selected_indices = sorted_indices[num_features_to_remove:]
    pruned_data = df.iloc[:, selected_indices]
    # save the removed features
    removed_features.extend(sorted_indices[:num_features_to_remove])
    return pruned_data, selected_indices

def reset_weights(m):
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [ ]:
# concatenate the train and validation sets
X_train_final = pd.concat([X_train, X_val])
y_train_final = np.concatenate([y_train, y_val])

In [ ]:
num_epochs = 3
num_features_to_remove = 2541

# load the model
model = Net3()
model.apply(reset_weights)

# Initialize the list for feature selection
removed_features = []

# RFE Loop
for rfe_iter in range(num_features_to_remove):

    model.to(device)

    # Start print
    print('===============================')
    print(f'RFE iteration {rfe_iter+1}/{num_features_to_remove}')
    print(f'Number of features: {X_train_final.shape[1]}')
    print('===============================')

    # convert the data into PyTorch tensors
    T_X_train_final = torch.from_numpy(X_train_final.values).float()

    # convert the target to a torch tensor
    T_y_train_final = torch.squeeze(torch.from_numpy(y_train_final)).long()

    # create Tensor datasets
    train_final_ds = data_utils.TensorDataset(T_X_train_final, T_y_train_final)

    # define the dataloader
    train_final_dl = data_utils.DataLoader(train_final_ds, batch_size=32, shuffle=True)

    train_losses, val_losses, val_accs = [], [], []
    for epoch in range(epochs):
        model, train_loss = train(model, train_final_dl, optimizer, criterion)
        print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} |')

    # Calculate gradients for feature importance
    gradients = calculate_gradients(model, T_X_train_final, T_y_train_final)

    # Perform feature selection based on gradients and update model architecture as needed
    X_train_final, _ = remove_least_important_features(X_train_final, gradients, removed_features, num_features_to_remove=10)

    # Update model architecture
    model.fc1 = nn.Linear(X_train_final.shape[1], 100)

    # Reset model weights
    model.apply(reset_weights)

    # Update the number of input features
    model.fc1.in_features = X_train.shape[1]

    print(np.array(df.columns.values.tolist())[removed_features])

# Print the removed features
print('===============================')
print('Removed features:')
print(np.array(df.columns.values.tolist())[removed_features])

Reset trainable parameters of layer = Linear(in_features=25511, out_features=100, bias=True)
Reset trainable parameters of layer = Linear(in_features=100, out_features=2, bias=True)
RFE iteration 1/100
Number of features: 25511
| Epoch: 01 | Train Loss: 37.162 |
| Epoch: 02 | Train Loss: 37.342 |
| Epoch: 03 | Train Loss: 37.256 |
Reset trainable parameters of layer = Linear(in_features=25501, out_features=100, bias=True)
Reset trainable parameters of layer = Linear(in_features=100, out_features=2, bias=True)
['ENSG00000039139' 'ENSG00000073670' 'ENSG00000151414' 'ENSG00000009413'
 'ENSG00000224892' 'ENSG00000180329' 'ENSG00000120798' 'ENSG00000072657'
 'ENSG00000240305' 'ENSG00000228224']
RFE iteration 2/100
Number of features: 25501
| Epoch: 01 | Train Loss: 30.152 |
| Epoch: 02 | Train Loss: 29.987 |
| Epoch: 03 | Train Loss: 29.607 |
Reset trainable parameters of layer = Linear(in_features=25491, out_features=100, bias=True)
Reset trainable parameters of layer = Linear(in_features=

In [ ]:
# save the removed features
np.savetxt('removed_features.csv', np.array(df.columns.values.tolist())[removed_features], delimiter=',', fmt='%s')
np.savetxt('removed_features_indices.csv', removed_features, delimiter=',', fmt='%s')